In [4]:
import math
from nltk.corpus import stopwords
from keras import backend as K
from keras.models import Model
from keras.optimizers import Adam
from keras.layers import Input, Dropout, Dense, concatenate, GRU, Embedding, Flatten, Activation
from keras.utils import pad_sequences
from keras.preprocessing.text import Tokenizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.pipeline import FeatureUnion
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import numpy as np
from datetime import datetime
start_real = datetime.now()


# from keras.layers import Bidirectional
# set seed
np.random.seed(123)

# Defining RMSLE Error Functions #
This will be used to check model accuracy after preductions are made on the train/test datasets

In [5]:
def rmsle(Y, Y_pred):
    assert Y.shape == Y_pred.shape
    return np.sqrt(np.mean(np.square(Y_pred - Y)))

# Loading Datasets #
1. Loading Data
2. Cleaning Datasets
3. Splitting Data into Test/Train

In [6]:
train_df = pd.read_table('Dataset/train.tsv')
test_df = pd.read_table('Dataset/test.tsv')
print(train_df.shape, test_df.shape)

(1482535, 8) (693359, 7)


### Pre-processing & Cleaning Datasets

Removing low prices, anything below 3. Postinggs below 3 are likely to be an error. Removing them helps the models.

In [7]:
# remove low prices
train_df = train_df.drop(train_df[(train_df.price < 3.0)].index)
train_df.shape

(1481661, 8)

Removing Stop-words from the dataset

In [10]:
stop = stopwords.words('english')
train_df["item_description"].fillna(value='No description yet', inplace=True)
train_df['item_description'] = train_df['item_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
train_df["name"].fillna(value="missing", inplace=True)
train_df['name'] = train_df['name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

test_df["item_description"].fillna(value='No description yet', inplace=True)
test_df['item_description'] = test_df['item_description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
test_df["name"].fillna(value="missing", inplace=True)
test_df['name'] = test_df['name'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard great condition works like came ...
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top hint lace key hole back! The pale...
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New tags. Leather horses. Retail [rm] each. St...
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete certificate authenticity


The length of the description, that is the raw number of words used, does have some correlation with price. The RNN might find this out on it's own, but since a max depth is used to save computations, it does not always know. Description length clearly helps the model, name length maybe not so much. Does not hurt the models so leaving name length in.

In [11]:
# get name and description lengths
def wordCount(text):
    try:
        if text == 'No description yet':
            return 0
        else:
            text = text.lower()
            words = [w for w in text.split(" ")]
            return len(words)
    except:
        return 0

train_df['desc_len'] = train_df['item_description'].apply(lambda x: wordCount(x))
test_df['desc_len'] = test_df['item_description'].apply(lambda x: wordCount(x))
train_df['name_len'] = train_df['name'].apply(lambda x: wordCount(x))
test_df['name_len'] = test_df['name'].apply(lambda x: wordCount(x))
train_df.head()

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len,name_len
0,0,MLB Cincinnati Reds T Shirt Size XL,3,Men/Tops/T-shirts,NaN,10.0,1,No description yet,0,7
1,1,Razer BlackWidow Chroma Keyboard,3,Electronics/Computers & Tablets/Components & P...,Razer,52.0,0,This keyboard great condition works like came ...,21,4
2,2,AVA-VIV Blouse,1,Women/Tops & Blouses/Blouse,Target,10.0,1,Adorable top hint lace key hole back! The pale...,16,2
3,3,Leather Horse Statues,1,Home/Home Décor/Home Décor Accents,NaN,35.0,1,New tags. Leather horses. Retail [rm] each. St...,22,3
4,4,24K GOLD plated rose,1,Women/Jewelry/Necklaces,NaN,44.0,0,Complete certificate authenticity,3,4


Here we split the category_name into 3 parts. The RNN models can get more information this way.

In [12]:
# split category name into 3 parts
def split_cat(text):
    try:
        return text.split("/")
    except:
        return ("No Label", "No Label", "No Label")


train_df['subcat_0'], train_df['subcat_1'], train_df['subcat_2'] = \
    zip(*train_df['category_name'].apply(lambda x: split_cat(x)))
test_df['subcat_0'], test_df['subcat_1'], test_df['subcat_2'] = \
    zip(*test_df['category_name'].apply(lambda x: split_cat(x)))

The brand name data is sparse, missing over 600,000 values. This gets *some* of those values back by checking their names. An *exact* name match against all_brand names will help finding some of the missing values. We can be pretty confident in these. At the other extreme, we can search for *any* matches throughout all words in name. This finds over 200,000 but a lot of these are incorrect. Can land somewhere in the middle by either keeping cases or trimming out some of the 5000 brand names.

For example, PINK is a brand by victoria secret. If we remove case, then almost all *pink* items are labeled as PINK brand. The other issue is that some of the "brand names" are not brands but really categories like "Boots" or "Keys". 

Currently, checking every word in name of a case-sensitive match does best. This gets around 137,000 finds while avoiding the problems with brands like PINK.

In [14]:
# attempt to find missing brand names
# train_df['name'] = train_df.name.str.lower()
# train_df['brand_name'] = train_df.brand_name.str.lower()
# test_df['name'] = test_df.name.str.lower()
# test_df['brand_name'] = test_df.brand_name.str.lower()
full_set = pd.concat([train_df, test_df])
all_brands = set(full_set['brand_name'].values)
train_df.brand_name.fillna(value="missing", inplace=True)
test_df.brand_name.fillna(value="missing", inplace=True)

# get to finding!
premissing = len(train_df.loc[train_df['brand_name'] == 'missing'])


def brandfinder(line):
    brand = line[0]
    name = line[1]
    namesplit = name.split(' ')
    if brand == 'missing':
        for x in namesplit:
            if x in all_brands:
                return name
    if name in all_brands:
        return name
    return brand


train_df['brand_name'] = train_df[['brand_name', 'name']].apply(brandfinder, axis=1)
test_df['brand_name'] = test_df[['brand_name', 'name']].apply(brandfinder, axis=1)
found = premissing-len(train_df.loc[train_df['brand_name'] == 'missing'])
print(found)

137314


In [23]:
# Scale target variable to log.
train_df["target"] = np.log1p(train_df.price)

# Split training examples into train/dev examples.
train, test = train_test_split(train_df, random_state=123, train_size=0.8)

# Calculate number of train/dev/test examples.
n_trains = train.shape[0]
n_devs = test.shape[0]
n_tests = test_df.shape[0]
print("Training on", n_trains, "examples")
print("Validating on", n_devs, "examples")
print("Testing on", n_tests, "examples")

Training on 938780 examples
Validating on 234695 examples
Testing on 693359 examples


# RNN Model fitting:
1. Preprocessing data
2. Define RNN model
3. Fitting RNN model on training examples
4. Evaluating RNN model on dev examples
5. Make prediction for test data using RNN model

In [24]:
# Concatenate train - dev - test data for easy to handle
full_df = pd.concat([train, test])

In [25]:
full_df

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len,name_len,subcat_0,subcat_1,subcat_2,target
295549,295549,Ava Anderson,1,Women/Jeans/Straight Leg,missing,6.0,1,Ava Anderson,2,2,Women,Jeans,Straight Leg,1.945910
297540,297540,Kendra Scott Jackie Necklace,2,Women/Jewelry/Necklaces,Kendra Scott,79.0,0,Kendra Scott Necklace / Gunmetal/ NWOT,6,4,Women,Jewelry,Necklaces,4.382027
583443,583443,FreeShip Striped Tee Brandy Style,2,Women/Tops & Blouses/T-Shirts,Brandy Melville,6.0,1,Super soft Brandy Melville like material Maroo...,16,5,Women,Tops & Blouses,T-Shirts,1.945910
978262,978262,Alo yoga vitality leggings,1,"Women/Athletic Apparel/Pants, Tights, Leggings",Alo,64.0,0,"Size small. Never worn, new tags Retails [rm]",8,4,Women,Athletic Apparel,"Pants, Tights, Leggings",4.174387
727950,727950,Girls 12/24 Mo John Deere Socks Baby 4,1,Kids/Girls 0-24 Mos/Accessories,missing,8.0,1,New 4 pairs John Deere socks Size 12/24 months...,14,8,Kids,Girls 0-24 Mos,Accessories,2.197225
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
543809,543809,Colourpop Liquid Matte Lipstick,2,Beauty/Makeup/Lips,ColourPop Cosmetics,9.0,0,Shade called perky. Never used,5,4,Beauty,Makeup,Lips,2.302585
541239,541239,Nike softball bag,2,Sports & Outdoors/Team Sports/Baseball & Softball,Nike,24.0,0,Black softball bag multiple pockets.,5,3,Sports & Outdoors,Team Sports,Baseball & Softball,3.218876
279365,279365,Nike Adult Shin Guards,2,Sports & Outdoors/Team Sports/Soccer,Nike,13.0,0,Size M Only worn Price firm,6,4,Sports & Outdoors,Team Sports,Soccer,2.639057
816880,816880,VS Pink Lace Bralette Highneck,1,Women/Underwear/Bras,Victoria's Secret,37.0,0,"New tags. Size large, black push style lace br...",51,5,Women,Underwear,Bras,3.637586


### Fill missing data
Note that replacing 'No description yet' with "missing" helps the model a bit by treating it the same as the NA values

In [38]:
# Filling missing values
def fill_missing_values(df):
    df["category_name"].fillna(value="missing", inplace=True)
    df["brand_name"].fillna(value="missing", inplace=True)
    df["item_description"].fillna(value="missing", inplace=True)
    df["item_description"].replace('No description yet', "missing", inplace=True)
    return df


print("Filling missing data...")
full_df = fill_missing_values(full_df)
print(full_df.loc[:,"category_name"])

Filling missing data...
295549                             Women/Jeans/Straight Leg
297540                              Women/Jewelry/Necklaces
583443                        Women/Tops & Blouses/T-Shirts
978262       Women/Athletic Apparel/Pants, Tights, Leggings
727950                      Kids/Girls 0-24 Mos/Accessories
                                ...                        
543809                                   Beauty/Makeup/Lips
541239    Sports & Outdoors/Team Sports/Baseball & Softball
279365                 Sports & Outdoors/Team Sports/Soccer
816880                                 Women/Underwear/Bras
509510                                   Beauty/Makeup/Lips
Name: category_name, Length: 1173475, dtype: object


### Encoding Categorical Data

In [39]:
print("Processing categorical data...")
le = LabelEncoder()
# full_df.category = full_df.category_name
le.fit(full_df.category_name)
full_df['category'] = le.transform(full_df.category_name)

le.fit(full_df.brand_name)
full_df.brand_name = le.transform(full_df.brand_name)

le.fit(full_df.subcat_0)
full_df.subcat_0 = le.transform(full_df.subcat_0)

le.fit(full_df.subcat_1)
full_df.subcat_1 = le.transform(full_df.subcat_1)

le.fit(full_df.subcat_2)
full_df.subcat_2 = le.transform(full_df.subcat_2)

# del le

Processing categorical data...


### Pre-Processing Text Data

Transforming Text Data into Tokens & then Sequences of numbers would help in fitting the RNN model

In [45]:
print("Transforming text data to sequences...")
raw_text = np.hstack([full_df["item_description"].str.lower(), full_df["name"].str.lower(), full_df["category_name"].str.lower()])

print("Fitting tokenizer...")
tok_raw = Tokenizer()
tok_raw.fit_on_texts(raw_text)

print("Transforming text to sequences...")
full_df['seq_item_description'] = tok_raw.texts_to_sequences(full_df["item_description"].str.lower())
full_df['seq_name'] = tok_raw.texts_to_sequences(full_df["name"].str.lower())
# full_df['seq_category'] = tok_raw.texts_to_sequences(full_df.category_name.str.lower())

del tok_raw

Transforming text data to sequences...
Fitting tokenizer...
Transforming text to sequences...


In [48]:
display(raw_text)
display(full_df.head())

array(['ava anderson', 'kendra scott necklace / gunmetal/ nwot',
       'super soft brandy melville like material maroon white stripes never worn brand new drapes flowy, flattering',
       ..., 'sports & outdoors/team sports/soccer',
       'women/underwear/bras', 'beauty/makeup/lips'], dtype=object)

,train_id,name,item_condition_id,category_name,brand_name,price,shipping,item_description,desc_len,name_len,subcat_0,subcat_1,subcat_2,target,category,seq_item_description,seq_name
295549,295549,Ava Anderson,1,Women/Jeans/Straight Leg,101042,6.0,1,Ava Anderson,2,2,10,57,726,1.945910,1150,"[7759, 8321]","[7759, 8321]"
297540,297540,Kendra Scott Jackie Necklace,2,Women/Jewelry/Necklaces,44145,79.0,0,Kendra Scott Necklace / Gunmetal/ NWOT,6,4,10,58,530,4.382027,1154,"[959, 900, 273, 4040, 307]","[959, 900, 10591, 273]"
583443,583443,FreeShip Striped Tee Brandy Style,2,Women/Tops & Blouses/T-Shirts,16959,6.0,1,Super soft Brandy Melville like material Maroo...,16,5,10,104,749,1.945910,1233,"[89, 138, 596, 650, 42, 208, 905, 53, 1184, 27...","[1299, 802, 262, 596, 172]"
978262,978262,Alo yoga vitality leggings,1,"Women/Athletic Apparel/Pants, Tights, Leggings",7725,64.0,0,"Size small. Never worn, new tags Retails [rm]",8,4,10,5,569,4.174387,1110,"[3, 36, 27, 19, 2, 54, 376, 9]","[6278, 652, 6687, 33]"
727950,727950,Girls 12/24 Mo John Deere Socks Baby 4,1,Kids/Girls 0-24 Mos/Accessories,101042,8.0,1,New 4 pairs John Deere socks Size 12/24 months...,14,8,4,46,5,2.197225,603,"[2, 28, 381, 2421, 6715, 457, 3, 146, 113, 357...","[78, 146, 113, 3301, 2421, 6715, 457, 153, 28]"


### Define constants to use when define RNN model
Note the comments next to the first few lines indicate the longest entry in that column. Just for reference.

In [49]:
MAX_NAME_SEQ = 10  # 17
MAX_ITEM_DESC_SEQ = 75  # 269
MAX_CATEGORY_SEQ = 8  # 8
MAX_TEXT = np.max([
    np.max(full_df["seq_name"].max()),
    np.max(full_df["seq_item_description"].max()),
    #     np.max(full_df.seq_category.max()),
]) + 100
MAX_CATEGORY = np.max(full_df["category"].max()) + 1
MAX_BRAND = np.max(full_df["brand_name"].max()) + 1
MAX_CONDITION = np.max(full_df["item_condition_id"].max()) + 1
MAX_DESC_LEN = np.max(full_df["desc_len"].max()) + 1
MAX_NAME_LEN = np.max(full_df["name_len"].max()) + 1
MAX_SUBCAT_0 = np.max(full_df["subcat_0"].max()) + 1
MAX_SUBCAT_1 = np.max(full_df["subcat_1"].max()) + 1
MAX_SUBCAT_2 = np.max(full_df["subcat_2"].max()) + 1

### Getting Data for RNN Model

In [50]:
# Converting the datasets from a Pandas dataframe to a dictionary to train the RNN models...

def get_rnn_data(dataset):
    X = {
        'name': pad_sequences(dataset.seq_name, maxlen=MAX_NAME_SEQ),
        'item_desc': pad_sequences(dataset.seq_item_description, maxlen=MAX_ITEM_DESC_SEQ),
        'brand_name': np.array(dataset.brand_name),
        'category': np.array(dataset.category),
        #         'category_name': pad_sequences(dataset.seq_category, maxlen=MAX_CATEGORY_SEQ),
        'item_condition': np.array(dataset.item_condition_id),
        'num_vars': np.array(dataset[["shipping"]]),
        'desc_len': np.array(dataset[["desc_len"]]),
        'name_len': np.array(dataset[["name_len"]]),
        'subcat_0': np.array(dataset.subcat_0),
        'subcat_1': np.array(dataset.subcat_1),
        'subcat_2': np.array(dataset.subcat_2),
    }
    return X


train = full_df[:n_trains]
test = full_df[n_trains:n_trains+n_devs]
# test = full_df[n_trains+n_devs:]

X_train = get_rnn_data(train)
Y_train = train["target"].values.reshape(-1, 1)

X_test = get_rnn_data(test)
Y_test = test["target"].values.reshape(-1, 1)

# X_test = get_rnn_data(test)

### Defining the RNN Model

In [55]:
# set seed again in case testing models adjustments by looping next 2 blocks
np.random.seed(123)


def new_rnn_model(lr=0.001, decay=0.0):
    # Inputs
    name = Input(shape=[X_train["name"].shape[1]], name="name")
    item_desc = Input(shape=[X_train["item_desc"].shape[1]], name="item_desc")
    brand_name = Input(shape=[1], name="brand_name")
#     category = Input(shape=[1], name="category")
#     category_name = Input(shape=[X_train["category_name"].shape[1]], name="category_name")
    item_condition = Input(shape=[1], name="item_condition")
    num_vars = Input(shape=[X_train["num_vars"].shape[1]], name="num_vars")
    desc_len = Input(shape=[1], name="desc_len")
    name_len = Input(shape=[1], name="name_len")
    subcat_0 = Input(shape=[1], name="subcat_0")
    subcat_1 = Input(shape=[1], name="subcat_1")
    subcat_2 = Input(shape=[1], name="subcat_2")

    # Embeddings layers (adjust outputs to help model)
    emb_name = Embedding(MAX_TEXT, 20)(name)
    emb_item_desc = Embedding(MAX_TEXT, 60)(item_desc)
    emb_brand_name = Embedding(MAX_BRAND, 10)(brand_name)
#     emb_category_name = Embedding(MAX_TEXT, 20)(category_name)
#     emb_category = Embedding(MAX_CATEGORY, 10)(category)
    emb_item_condition = Embedding(MAX_CONDITION, 5)(item_condition)
    emb_desc_len = Embedding(MAX_DESC_LEN, 5)(desc_len)
    emb_name_len = Embedding(MAX_NAME_LEN, 5)(name_len)
    emb_subcat_0 = Embedding(MAX_SUBCAT_0, 10)(subcat_0)
    emb_subcat_1 = Embedding(MAX_SUBCAT_1, 10)(subcat_1)
    emb_subcat_2 = Embedding(MAX_SUBCAT_2, 10)(subcat_2)

    # rnn layers (GRUs are faster than LSTMs and speed is important here)
    rnn_layer1 = GRU(16)(emb_item_desc)
    rnn_layer2 = GRU(8)(emb_name)
#     rnn_layer3 = GRU(8) (emb_category_name)

    # main layers
    main_l = concatenate([
        Flatten()(emb_brand_name)  
        # , Flatten() (emb_category)
        , Flatten()(emb_item_condition)
        , Flatten()(emb_desc_len)
        , Flatten()(emb_name_len)
        , Flatten()(emb_subcat_0)
        , Flatten()(emb_subcat_1)
        , Flatten()(emb_subcat_2)
        , rnn_layer1
        , rnn_layer2  
        # , rnn_layer3
        , num_vars
    ])
    # (incressing the nodes or adding layers does not effect the time quite as much as the rnn layers)
    main_l = Dropout(0.1)(Dense(512, kernel_initializer='normal', activation='relu')(main_l))
    main_l = Dropout(0.1)(Dense(256, kernel_initializer='normal', activation='relu')(main_l))
    main_l = Dropout(0.1)(Dense(128, kernel_initializer='normal', activation='relu')(main_l))
    main_l = Dropout(0.1)(Dense(64, kernel_initializer='normal', activation='relu')(main_l))

    # the output layer.
    output = Dense(1, activation="linear")(main_l)

    model = Model([name, item_desc, brand_name, item_condition, num_vars, desc_len, name_len, subcat_0, subcat_1, subcat_2], output)

    optimizer = Adam(lr=lr, decay=decay)
    # (mean squared error loss function works as well as custom functions)
    model.compile(loss='mse', optimizer=optimizer)

    return model

model = new_rnn_model()
model.summary()
del model

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 brand_name (InputLayer)        [(None, 1)]          0           []                               
                                                                                                  
 item_condition (InputLayer)    [(None, 1)]          0           []                               
                                                                                                  
 desc_len (InputLayer)          [(None, 1)]          0           []                               
                                                                                                  
 name_len (InputLayer)          [(None, 1)]          0           []                               
                                                                                              

c:\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


### Fitting RNN model to train data

In [56]:
# Set hyper parameters for the model.
BATCH_SIZE = 512 * 3
epochs = 2

# Calculate learning rate decay.


def exp_decay(init, fin, steps): return (init/fin)**(1/(steps-1)) - 1


steps = int(len(X_train['name']) / BATCH_SIZE) * epochs
lr_init, lr_fin = 0.005, 0.001
lr_decay = exp_decay(lr_init, lr_fin, steps)

# Create model and fit it with training dataset.
rnn_model = new_rnn_model(lr=lr_init, decay=lr_decay)
rnn_model.fit(X_train, Y_train, epochs=epochs, batch_size=BATCH_SIZE,validation_data=(X_test, Y_test), verbose=1,)

c:\Python311\Lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/2


c:\Python311\Lib\site-packages\keras\engine\functional.py:639: UserWarning: Input dict contained keys ['category'] which did not match any model input. They will be ignored by the model.
  if mask is None:


612/612 [==============================] - 202s 323ms/step - loss: 0.4217 - val_loss: 0.2122
Epoch 2/2
612/612 [==============================] - 226s 370ms/step - loss: 0.2217 - val_loss: 0.2005


### Evaluating RNN Model on Train & Test Data

In [58]:
# Evaluating Accuracy on Test Dataset
print("Evaluating the model on validation data...")
Y_test_preds_rnn = rnn_model.predict(X_test, batch_size=BATCH_SIZE)
print(" RMSLE error:", rmsle(Y_test, Y_test_preds_rnn))

Evaluating the model on validation data...
153/153 [==============================] - 10s 62ms/step
 RMSLE error: 0.4477298881579948


In [59]:
# Evaluating Accuracy on Training Dataset
print("Evaluating the model on validation data...")
Y_train_preds_rnn = rnn_model.predict(X_train, batch_size=BATCH_SIZE)
print(" RMSLE error:", rmsle(Y_train, Y_train_preds_rnn))

Evaluating the model on validation data...
612/612 [==============================] - 34s 55ms/step
 RMSLE error: 0.3799419746089792
